# Model Performance Evaluation

## Total Variation Distance

One metric to compare the performance of different models is to determine the similarity of each generated sequence to some reference (e.g. wildtype reference or closest sequence in training dataset), which yields a histogram of similarity values for each model. The total variation distance (TVD) between the histograms of different models can then be calculated as a notion of model similarity. When using the wildtype sequence as reference for the similarity, models can even be compared to the training dataset.

Running the TVD computation is implemented in ```scripts/tvd_matrix.py```. Computation for the MID1 and IRed datasets is supported out of the box, simply modify the paths in ```res_dirs``` at the bottom to point to the files containing the generated sequences of your models.
When adding your own dataset, note that the script assumes that the loader class implements the ```loader.reference``` attribute which contains a string with the wildtype reference sequence.

If the obtained values in the tvd matrix exhibit odd behavior (e.g. all very small / large), one potential error source might be the choice of bins for determining the histograms. For the linear metrics, Blosum and One-hot similarity, ```bins``` specifies the number of equally spaced bins to choose. For the inverse metrics (Hamming similarity and Blosum-weighted Hamming similarity), it specifies the distance in terms of (weighted) insertions between two bin edges. The bin interval will always range from smallest to highest similarity value.

In [ ]:
!python3 ../scripts/tvd_matrix.py --use_wildtype -l 290 -d ired -o ./tvd_matrix.png

## ESM-2 latent space

The embeddings of generated and real sequences obtained from Evolutionary Scale Modelling 2 (ESM-2) can be used to assess the relative performance of different models. Additionally, the perplexity of ESM-2 can be employed as a proxy for biological feasibility of the sequences.

First, we need to specify a handle for each model and a path to where the sequences generated by it are stored. This is done as a ```dict``` like so:

In [ ]:
res_dirs = {
    "sedd": "./gen_data/sedd/ired.fasta",
    "dfm": "./gen_data/dfm/ired.fasta",
    "frozen": "./gen_data/frozen/1.5.1.-.fasta",
    "pretrained": "./gen_data/zymCTRL_results/1.5.1.-.fasta",
    "tiny": "./gen_data/tiny_model/1.5.1.-.fasta",
    "small": "./gen_data/small_model/1.5.1.-.fasta",
    "ft lr 8e-05": "./gen_data/zytune_untrunc/1.5.1.-.fasta",
    "ft lr 8e-06": "./gen_data/zymctrl_lr-06/1.5.1.-.fasta",
    "ft lr 8e-07": "./gen_data/zymctrl_lr-07/1.5.1.-.fasta",
    "ft lr 8e-08": "./gen_data/zymctrl_lr-08/1.5.1.-.fasta",
    "ft lr 8e-09": "./gen_data/zymctrl_lr-09/1.5.1.-.fasta",
    "potts gwg": "./gen_data/potts_adam_gwg_T_0.01_lr_0.001_unified.fasta",
    "random informed": "./gen_data/random_trained.fasta",
    "random": "./gen_data/random.fasta",
    }

Then, the data has to be loaded and ```n_per_dataset``` unique sequences can be sampled randomly.
In order to prevent tokenization issues with the ESM-2 tokenizer, we remove any numbers and other unknown characters from the sequences. Depending on whether you choose to extend the alphabet of some models with additional characters that are not known by the ESM-2 tokenizer, you would have to remove these aswell.

In [ ]:
import numpy as np
from genzyme.data import loaderFactory
from genzyme.evaluation.embedding import ESM, EmbeddingStats

n_per_dataset = 600
l = 97
seed = 7

seqs = []
labs = []

np.random.seed(seed)

for i, path in enumerate(res_dirs.values()):
    with open(path, "r") as file:
        lines = [s.strip().replace("[UNK]", "").replace("X", "") for s in file.readlines() if not s.startswith('>')]
        lines = np.unique([''.join([pos for pos in seq if not pos.isdigit()]) for seq in lines])
        
    lines = np.unique([x[:l] for x in lines if len(x) >= l])

    if len(lines) > n_per_dataset:
        lines = np.random.choice(lines, n_per_dataset, replace=False)

    seqs.extend(lines)
    labs.extend(len(lines)* [i])

train_dat = loaderFactory()

Now, the training data can be loaded aswell using a dataloader.

In [ ]:
train_dat = loaderFactory()
train_dat.load('ired')
train_dat._replace("*")

lines = train_dat.get_data()[np.random.choice(len(lines), n_per_dataset, replace=False)]
labs.extend([i+1]*len(lines))
seqs.extend(lines)

Compute the embeddings, negative log-likelihoods and perplexities using the ESM-2 model. As the model can handle sequences of different lengths, there's no need to perform truncation / filtering.

In [ ]:
embs = ESM()
embeddings, nll, ppl = embs.get_embeddings(seqs)

Average perplexity and NLL per model

In [ ]:
avg_nll = {}
avg_ppl = {}
for l in np.unique(labs):
    avg_nll[(list(res_dirs.keys())+["train data"])[l]] = np.mean(nll[np.array(labs) == l])
    avg_ppl[(list(res_dirs.keys())+["train data"])[l]] = np.mean(ppl[np.array(labs) == l])

print(avg_nll)
print(avg_ppl)

In [ ]:
stats = EmbeddingStats(np.array(labs), np.array(embeddings), label_names = list(res_dirs.keys())+["train data"])

Get the average silhouette score of the clusters defined by the model membership

In [ ]:
stats.silhouette()

Plot the embeddings using a PCA or UMAP projection. The default is a PCA projection colored by model.

In [ ]:
import matplotlib.pyplot as plt

_, ax = stats.plot_embeddings()
ax.set_title('ESM-2 embeddings (PCA)')
plt.show()

In [ ]:

_, ax = stats.plot_embeddings(color = ppl, col_name = "perplexity")
ax.set_title('ESM-2 embeddings (PCA)')
plt.show()

In [ ]:
_, ax = stats.plot_embeddings(umap=True)
ax.set_title('ESM-2 embeddings (UMAP)')
plt.show()

In [ ]:
_, ax = stats.plot_embeddings(color = ppl, col_name = "perplexity", umap=True)
ax.set_title('ESM-2 embeddings (UMAP)')
plt.show()